In [236]:
# Imports
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Irrelevant variables
dispos = {
    "Berkshire Roots" : "https://dutchie.com/dispensary/berkshire-roots-east-boston",
    "Mission Brookline" : "https://dutchie.com/dispensary/mission-brookline",
    "NETA" : "https://dutchie.com/dispensary/neta-brookline",
    "Western Front" : "https://dutchie.com/dispensary/western-front1",
    "MedMen" : "https://dutchie.com/dispensary/medmen-fenway",
    "Rise" : "https://dutchie.com/dispensary/greenstar-herbals-inc-chelsea",
    "Happy Valley" : "https://dutchie.com/dispensary/happy-valley-east-boston",
    "Pure Oasis" : "https://dutchie.com/dispensary/pure-oasis1",
    "Ayr" : "https://dutchie.com/dispensary/ayr-dispensary-back-bay"
}

filters = {
    "Concentrate, Ascending Price" : "/products/concentrates?sortby=pricelowtohigh"
}


In [237]:
# Session options
options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)


In [238]:
# Product class w/ attrs.
class Product:
  def __init__(self, name, company, type, thc, weight, price, oldprice, discount, dispo):
    self.name = name
    self.company = company
    self.type = type
    self.thc = thc
    self.weight = weight
    self.price = price
    self.oldprice = oldprice
    self.discount = discount
    self.dispo = dispo
    

In [239]:
# Scrapes URL and returns the main element containing products and prod info
def scrapeURL(driver, dispo):

    # Connect to URL
    filterURL = dispos[dispo] + filters["Concentrate, Ascending Price"]
    driver.get(filterURL)
    driver.implicitly_wait(1.5)
    #time.sleep(5)

    # Age verification
    
    try:
        driver.find_element(By.CLASS_NAME, "age-confirmation-modal__StyledButton-di8wrk-0").click()
    except:
        print("")


    # Smooth scroll to scrub ALL data
    scheight = .1
    while scheight < 9.9:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/%s);" % scheight)
        scheight += .01

    # Main element with all products
    main_element = driver.find_element(By.TAG_NAME, "main")
    
    # Searching for the product lists
    element_products = main_element.find_element(By.CLASS_NAME, "product-list__Container-sc-1arkwfu-1")
    element_div = element_products.find_element(By.TAG_NAME, "div")
    element_prod_list = element_div.find_elements(By.CLASS_NAME, "mobile-product-list-item__Container-zxgt1n-0")
    prod_list = element_prod_list[1:len(element_prod_list)-1]

    # Parsing the list into organized Product class
    products = []
    for element in prod_list:
        data = element.text.split("\n")
        if (len(data) == 7):   # Discounted Product
            p = Product(data[0], data[1], data[2], data[2], data[3], data[5], data[4], data[6], dispo)
            products.append(p)
        elif (len(data) == 6): # Staff Pick Product
            p = Product(data[1], data[2], data[3], data[3], data[4], data[5], data[5], "0%", dispo)
            products.append(p)
        elif (len(data) == 5): # Regular Product
            p = Product(data[0], data[1], data[2], data[2], data[3], data[4], data[4], "0%", dispo)
            products.append(p)
    
    print(len(products), "products scraped from", dispo)
    return products

In [242]:
products = []
for dispo in dispos:
    dispo_products = scrapeURL(driver, dispo)
    products.extend(dispo_products)


84 products scraped from Berkshire Roots

33 products scraped from Mission Brookline

26 products scraped from NETA

6 products scraped from Western Front

6 products scraped from MedMen

12 products scraped from Rise

20 products scraped from Happy Valley

11 products scraped from Pure Oasis

65 products scraped from Ayr


In [251]:
p = products

print(len(p))

for pr in p:
    if pr.discount != "na":
        print(pr.name, pr.price, pr.weight, pr.dispo)

263
Snowdog Kief | 1g - Essentials $19.96 1g - Berkshire Roots
Deep 6 #55 LIVE Badder | .5g - Signature $25.46 .5g - Berkshire Roots
Grand Daddy Purple LIVE Badder | .5g - Signature $25.46 .5g - Berkshire Roots
Snowdog LIVE Sugar | .5g - Signature $25.46 .5g - Berkshire Roots
Zombie Virus LIVE Badder | .5g - Signature $25.46 .5g - Berkshire Roots
Grand Daddy Purple Badder | .5g - Signature $27.16 .5g - Berkshire Roots
Zombie Virus Badder | .5g - Signature $27.16 .5g - Berkshire Roots
Great White Shark Sugar Wax | 1g - Signature $47.16 1g - Berkshire Roots
Zombie Virus Badder | 1g - Signature $47.16 1g - Berkshire Roots
THC Distillate Applicator| 1g - Basics $48.97 1g - Berkshire Roots
Staff Pick .5g - Nova Farms Mission Brookline
D95 (Hybrid) - Shatter, 0.5g $21.00 .5g - NETA
Dread Bread (Sativa) - Kief, 1g $21.00 1g - NETA
Hybrid Blend (Hybrid) - Kief, 1g $21.00 1g - NETA
Indica Blend (Indica) - Kief, 1g $21.00 1g - NETA
Maui Wowie (Sativa) - Shatter, 0.5g $21.00 .5g - NETA
Milkbone (